## Hyperparameter Optimization For XGBoost using GridSearchCV

In [13]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

In [14]:
# Read the Dataset
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [23]:
print(data.describe)

AttributeError: 'numpy.ndarray' object has no attribute 'desc'

In [24]:
print(type(data['data']))
print(type(data['target']))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
print(data['data'].shape)
print(data['target'].shape)

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df

In [ ]:
df['PRICE'] = data.target
df

In [ ]:
df.describe(include='all')

In [ ]:
df.info()

In [ ]:
# Correlation

import seaborn as sns
import matplotlib.pyplot as plt

#get correlations of each pair of features in the data
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20, 20))

#plot heat map
sns.heatmap(df[top_corr_features].corr(), annot=True, cmap="RdYlGn")

In [ ]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X,
                                                                    y,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    random_state=0)
print(y.shape[0])
print(y_train.shape[0])
print(y_test.shape[0])

In [ ]:
import xgboost as xgb
xgreg = xgb.XGBRegressor()
xgreg.fit(X_train, y_train)

In [ ]:
# r2_score, coefficient of determination
score = xgreg.score(X_train, y_train)
print('r2_score on training data =', score)

In [ ]:
from sklearn import metrics

print('r2_score on training data =', metrics.r2_score(y_true=y_train,
                                                      y_pred=xgreg.predict(X_train),
                                                      multioutput='variance_weighted'))

In [ ]:
print('r2_score on test data =', metrics.r2_score(y_true=y_test,
                                                  y_pred=xgreg.predict(X_test),
                                                  multioutput='variance_weighted'))

In [ ]:
# Hyperparameter Optimization

xgreg = xgb.XGBRegressor()

# Default-Run of default-hyperparameters
parameters = {'learning_rate': [0.3],
              'max_depth': [6],
              'min_child_weight': [1],
              'n_estimators': [100]}


reg_xgb = model_selection.GridSearchCV(estimator=xgreg,
                                       param_grid=parameters,
                                       n_jobs=-1,
                                       cv=3,
                                       refit=True)

reg_xgb.fit(X_train, y_train)

In [ ]:
print(reg_xgb.best_params_)
print(reg_xgb.best_score_)
print(reg_xgb.best_estimator_)

In [ ]:
# 1st-Run for best hyperparameters
parameters = {'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
              'max_depth': [2, 4, 6, 8, 10],
              'min_child_weight': [1, 10, 20, 30, 40, 50],
              'n_estimators': [100, 200, 300, 400, 500]}


reg_xgb = model_selection.GridSearchCV(estimator=xgreg,
                                       param_grid=parameters,
                                       n_jobs=-1,
                                       cv=3,
                                       refit=True)

reg_xgb.fit(X_train, y_train)

In [ ]:
print(reg_xgb.best_params_)
print(reg_xgb.best_score_)
print(reg_xgb.best_estimator_)

In [ ]:
# 2nd-Run for best hyperparameters
parameters = {'learning_rate': [0.3, 0.35, 0.4, 0.45, 0.5],
              'max_depth': [2, 3, 4, 5, 6],
              'min_child_weight': [10, 15, 20, 25, 30],
              'n_estimators': [50, 100, 150, 200, 250]}


reg_xgb = model_selection.GridSearchCV(estimator=xgreg,
                                       param_grid=parameters,
                                       n_jobs=-1,
                                       cv=3,
                                       refit=True)

reg_xgb.fit(X_train, y_train)

In [ ]:
print(reg_xgb.best_params_)
print(reg_xgb.best_score_)
print(reg_xgb.best_estimator_)

In [ ]:
# 3rd-Run for best hyperparameters
parameters = {'learning_rate': [0.35, 0.375, 0.4, 0.425, 0.45],
              'max_depth': [3, 4, 5, 6, 7],
              'min_child_weight': [15, 18, 20, 22, 25],
              'n_estimators': [80, 90, 100, 110, 120]}


reg_xgb = model_selection.GridSearchCV(estimator=xgreg,
                                       param_grid=parameters,
                                       n_jobs=-1,
                                       cv=3,
                                       refit=True)

reg_xgb.fit(X_train, y_train)

In [ ]:
print(reg_xgb.best_params_)
print(reg_xgb.best_score_)
print(reg_xgb.best_estimator_)

In [ ]:
print('r2_score on test data =', metrics.r2_score(y_true=y_test,
                                                  y_pred=reg_xgb.predict(X_test),
                                                  multioutput='variance_weighted'))

In [ ]:
final_model = xgb.XGBRegressor(learning_rate=0.425,
                               max_depth=3,
                               min_child_weight=22,
                               n_estimators=120)

final_model.fit(X_train,y_train)
print('r2_score on train data =', metrics.r2_score(y_true=y_train,
                                                   y_pred=reg_xgb.predict(X_train),
                                                   multioutput='variance_weighted'))

print('r2_score on test data =', metrics.r2_score(y_true=y_test,
                                                  y_pred=reg_xgb.predict(X_test),
                                                  multioutput='variance_weighted'))